-ve sun angles - all other rows 0
no park trees included in shading
tree locations - 5 decimals
metric data - 3 decimals
seperate csv for trees

In [1]:
import pandas as pd
import numpy as np
import laspy
import json
import pptk
from math import *
from matplotlib import pyplot as plt
from matplotlib import path
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import KDTree
import alphashape
from shapely.geometry import Point
from scipy.spatial import ConvexHull
from pysolar.solar import *
import datetime
import csv
import pytz
import time
import logging
import re
import os
from pyproj import Transformer
from datetime import timedelta


# from ipywidgets import IntProgress
# from IPython.display import display

from multiprocessing import Pool


#HARDCODED VALUES

COLOR_DICT = {
    "WHITE" : [255,255,255],
    "RED" : [1,0,0],
    "GREEN" : [0,1,0],
    "BLUE" : [0,0,1],
    "YELLOW" : [255,255,0],
    "LIGHT_BLUE" : [0,255,255],
    "PINK" : [255,0,255]
}

# DATE_DICT = {
#     "YEAR" : 2021,
#     "MONTH" : 6,
#     "DAY" : 21
# }


def Create_Dataframe_fromLas(lasFilePath):
    las = laspy.read(lasFilePath)
    #point_format = las.point_format
    lidar_points = np.array((las.X,las.Y,las.Z,las.intensity,las.classification, las.return_number, las.number_of_returns)).transpose()
    lidar_df = pd.DataFrame(lidar_points)
    lidar_df[0] = lidar_df[0]/100
    lidar_df[1] = lidar_df[1]/100
    lidar_df[2] = lidar_df[2]/100
    lidar_df.columns = ['X', 'Y', 'Z', 'intens', 'class', 'return_number', 'number_of_returns']
    
    return lidar_df

def Extract_SRdata(SR_JSONarr):
    #Store all Raw Treepoints
    SR_RawTreePoints = []
    #Store all Tree points mapped to a Tree ID
    SR_TreeClusterDict = {}
    #Store Tree Heights
    TreeHeights_ClusterHashmap = {}
    #Store GroundZ Elevation
    GroundElevation_ClusterHashmap = {}
    #Store All Tree Ids
    JSONClusterID_SR = []

    for i in range(len(SR_JSONarr)):
        #get points and cluster ID
        points = SR_JSONarr[i]["SRpointsInfo"]["SpecificClusterSRpoints"]
        cluster_id = SR_JSONarr[i]['SRpointsInfo']['SRpointsTreeCluster']
        #Storing with Cluster ID hashmap prevents storing multiple points from the same tree, if Tree present
        if cluster_id not in SR_TreeClusterDict:
            SR_TreeClusterDict[cluster_id] = points
        else:
            for p in points: 
                SR_TreeClusterDict[cluster_id].append(p)
        #Tree Heights
        if cluster_id not in TreeHeights_ClusterHashmap:
            TreeHeights_ClusterHashmap[cluster_id] = SR_JSONarr[i]['TreeFoliageHeight']
        #GroundZvalue
        if cluster_id not in GroundElevation_ClusterHashmap:
            GroundElevation_ClusterHashmap[cluster_id] = SR_JSONarr[i]['GroundZValue']

        #Raw Points
        for j in points:
            SR_RawTreePoints.append(j)

    return SR_RawTreePoints, SR_TreeClusterDict, TreeHeights_ClusterHashmap, GroundElevation_ClusterHashmap

def Extract_MRdata(MR_JSONarr):
    ##Store all Raw Treepoints
    MR_RawTreePoints = []
    #Store all Tree points mapped to a Tree ID
    MR_TreeClusterDict = {}
    #Store Cluster Location
    TreeLocation_ClusterHashmap = {}
    
    for i in range(len(MR_JSONarr)):
        #get points and cluster ID
        points = MR_JSONarr[i]["ConvexHullDict"]["ClusterPoints"]
        cluster_id = MR_JSONarr[i]['ClusterID']
        #Storing with Cluster ID hashmap prevents storing multiple points from the same tree, if Tree present
        if cluster_id not in MR_TreeClusterDict:
            MR_TreeClusterDict[cluster_id] = points
        else:
            for p_mr in points: 
                MR_TreeClusterDict[cluster_id].append(p_mr)

        Tree_lat = MR_JSONarr[i]["PredictedTreeLocation"]["Latitude"]
        Tree_long = MR_JSONarr[i]["PredictedTreeLocation"]["Longitude"]

        if cluster_id not in TreeLocation_ClusterHashmap:
            TreeLocation_ClusterHashmap[cluster_id] = [Tree_lat,Tree_long]

        #Raw points
        for k in points:
            MR_RawTreePoints.append(k)
    
    return MR_RawTreePoints, MR_TreeClusterDict, TreeLocation_ClusterHashmap

def Get_SR_Treepoints_from_Hashmap(TreeClusterID, SR_TreeClusterDict, las_filename='25192'):
    return SR_TreeClusterDict[las_filename+"_"+str(TreeClusterID)]

def Get_MR_Treepoints_from_Hashmap(TreeClusterID, MR_TreeClusterDict, las_filename='25192'):
    return MR_TreeClusterDict[las_filename+"_"+str(TreeClusterID)]

def Get_FullTree_points(TreeClusterArr, SR_TreeClusterDict, MR_TreeClusterDict, Ground_Elevation = 0, Tree_Height = 0):
    Full_Tree = []
    for Tree_Id in TreeClusterArr:
        TreePoints_SR = Get_SR_Treepoints_from_Hashmap(Tree_Id,SR_TreeClusterDict)
        TreePoints_MR = Get_MR_Treepoints_from_Hashmap(Tree_Id,MR_TreeClusterDict)
        All_SingleTree_points = np.concatenate((TreePoints_MR,TreePoints_SR), axis=0)
        All_SingleTree_points = All_SingleTree_points*3.28 # Convert m to ft
        for p in All_SingleTree_points:
            Full_Tree.append(p - [0,0,Ground_Elevation+Tree_Height]) #Height Adjusted, default = 0
            
    return np.array(Full_Tree)

def Get_FullTreeCentroid(Full_Tree_points):
    centroid = np.mean(Full_Tree_points,axis=0)
    centroid[2] = 0

    return centroid

def Get_Shadow(points, az, amp):
    projected_points = []
    for point in points:
        sinAz = sin( radians( az + 180.0 ) )
        cosAz = cos( radians( az + 180.0 ) )
        tanAmp = tan( radians(amp) )
        pointGroundX = point[0] + ( ( point[2] / tanAmp ) *sinAz )
        pointGroundY = point[1] + ( ( point[2] / tanAmp ) *cosAz )
        pointGroundZ =  point[2] * 0
    
        projected_points.append([pointGroundX,pointGroundY,pointGroundZ])
    
    return np.array(projected_points)

def Get_ShadowCharacteristics(Full_Tree, TreeShadow):

    F_distance, F_point = Get_FurthestPointFromTreeCentroid(Full_Tree, TreeShadow)
    centroid = Get_FullTreeCentroid(Full_Tree)
    hull = ConvexHull(TreeShadow[:,:2])

    Shadow_Area = hull.area
    Shadow_length = np.linalg.norm(centroid - F_point) # Tree Centroid - Furthest point from Tree centroid on Tree shadow
    shadow_breadth = Shadow_Area/Shadow_length

    return Shadow_length, shadow_breadth, Shadow_Area

def Get_FurthestPointFromTreeCentroid(Full_Tree, TreeShadow):
    
    Shadow_KDTree = KDTree(TreeShadow)
    centroid = Get_FullTreeCentroid(Full_Tree)
    dist,ind = Shadow_KDTree.query(centroid.reshape(1,3),len(TreeShadow))
    
    Furtherst_Point_Distance = dist[0][-1]
    Furthest_Point_Vector = TreeShadow[ind[0][-1]]

    return Furtherst_Point_Distance, Furthest_Point_Vector

def Get_ShadowAlphaShape(ShadowPoints):
    return alphashape.alphashape(ShadowPoints, alpha=0)

# func v1
# def Get_TreeLocation(TreeID, TreeID_Location_dict, las_filename='25192'):
#     return TreeID_Location_dict[las_filename+"_"+str(TreeID)]
# func_v2
def Get_TreeLocation(json_dataBuffer):
    lat = json_dataBuffer['PredictedTreeLocation']['Latitude']
    lon = json_dataBuffer['PredictedTreeLocation']['Longitude']
    return lat,lon

def Get_TreeGroundElevation(TreeID, GroundElevation_ClusterDict, las_filename='25192'):
    return GroundElevation_ClusterDict[las_filename+"_"+str(TreeID)]
    
def Get_TreeHeight(TreeID, TreeHeights_ClusterDict, las_filename='25192'):
    return TreeHeights_ClusterDict[las_filename+"_"+str(TreeID)]

def Get_GroundElevation(BuildingLidarDict):
    arr = (np.concatenate(list(BuildingLidarDict.values())))
    #Get ground elevation from building points min(list(BuildingLidarDict.values())[0][:,2])
    return min(arr[:,2])

#Code to automate Obtaining Az and Amp
def Get_SunData(Tree_Latitude, Tree_Longitude, year, month, day, hour, minute):
    date = datetime.datetime(year, month, day, hour, minute, 0, 0, tzinfo=datetime.timezone.utc)
    date_est = date.astimezone(pytz.timezone('US/Eastern')).isoformat()
    az = get_azimuth(Tree_Latitude, Tree_Longitude, date)
    amp = get_altitude(Tree_Latitude, Tree_Longitude, date)

    return date_est, az, amp

def pptk_wrapperSimplePlot(PointsArr, Color):
    v = pptk.viewer(PointsArr, [Color]*len(PointsArr))
    v.set(show_grid=False)
    v.set(show_axis=False)
    v.set(bg_color = [0,0,0,0])
    v.set(point_size = 0.04)

def sample_polygon(V, eps=0.25):
    # samples polygon V s.t. consecutive samples are no greater than eps apart
    # assumes last vertex in V is a duplicate of the first
    M = np.ceil(np.sqrt(np.sum(np.diff(V, axis=0) ** 2, axis = 1)) / eps)
    Q = []
    for (m, v1, v2) in zip(M, V[: -1], V[1:]):
        Q.append(np.vstack([ \
            np.linspace(v1[0], v2[0], endpoint = False), \
            np.linspace(v1[1], v2[1], endpoint = False)]).T)
    Q = np.vstack(Q)

    return Q

def Read_GeoJSON(filepath):
    with open(filepath, 'rb') as fd:
        data = json.load(fd)

    return data

    

def Get_SampledBuildingFootprints(Buildingdata):
    BuildingFeature_Coords = [np.array(F['geometry']['coordinates'][0][0]) for F in Buildingdata['features']]
    SampledFootprint = np.vstack([sample_polygon(W) for W in BuildingFeature_Coords])
    SampledFootprint = np.c_[SampledFootprint, np.zeros(len(SampledFootprint))]

    return SampledFootprint

def Get_BuildingFootprint(GeoJSON_Filepath):
    BuidlingFootprintData = Read_GeoJSON(GeoJSON_Filepath)

    return Get_SampledBuildingFootprints(BuidlingFootprintData)

def Get_BuildingDataDict(BuildingFilePath, f, year=2017):
    with open(BuildingFilePath, 'rb') as fd:
        NYC_building_Footprint = json.load(fd)
    
    Vs_25192 = [np.array(F['geometry']['coordinates'][0][0]) for F in NYC_building_Footprint['features']]
    ### **Note** : NYC points already projected to state plane, No need for pyproj
    Ws_25192 = Vs_25192
    Total_Building_Count = len(NYC_building_Footprint['features'])
    #Building_Data_dict
    Building_coords_dict = {}
    B_ID = 0
    Building_Key = f[:-4] + "_" + str(year) + "_Building_" + str(B_ID) # initialized to 25192_2015_Building_0

    for building in range(Total_Building_Count):

        B_ID += 1
        Building_Key = f[:-4] + "_" + str(year) + "_Building_" + str(B_ID) # initialized to 25192_2015_Building_0
        building_coords = sample_polygon(Ws_25192[building]) #NYC_building_Footprint['features'][building]['geometry']['coordinates'][0][0]
        Building_coords_dict[Building_Key] = building_coords
    
    return Building_coords_dict

#NOTE: Function takes most time
def Get_BuildingsUnderEffectOfShadow(Building_coords_dict, TreeShadowPoints):
    ShadowAlphaShape = Get_ShadowAlphaShape(TreeShadowPoints)
    Affected_Building_IDarr = [] #Buildings Under the Direct Affect of the Shadow
    for B_ID, Sampled_B_coords in Building_coords_dict.items():
        bool_temp_list = []
        for point in Sampled_B_coords:
            tp = Point(point)
            bool_temp_list.append(tp.within(ShadowAlphaShape))
        #check if any points in the sampled footprint fall within the area of shadow
        if len(Sampled_B_coords[bool_temp_list]) > 0:
            Affected_Building_IDarr.append(B_ID)
    
    return Affected_Building_IDarr

def Get_BuildingsUnderEffectOfShadow_Rapid(Building_coords_dict, TreeShadowPoints):

    Affected_Building_IDarr = [] #Buildings Under the Direct Affect of the Shadow
    for B_ID, Sampled_B_coords in Building_coords_dict.items():
        bool_temp_list = []

        p = path.Path(TreeShadowPoints[:,:2])
        bool_temp_list = p.contains_points(Sampled_B_coords)
        # #check if any points in the sampled footprint fall within the area of shadow
        if len(Sampled_B_coords[bool_temp_list]) > 0:
            Affected_Building_IDarr.append(B_ID)
    
    return Affected_Building_IDarr

def Get_BuildingFootprintSampledCoords(BIDs_Arr, Building_coords_dict):

    labelled_BF_points = []
    for B_ID in BIDs_Arr:

        b_points = Building_coords_dict[B_ID]

        for p in b_points:
            labelled_BF_points.append(p)

    labelled_BF_points = np.array(labelled_BF_points) #shape : (Nx2)

    #reformat for plotting purposes
    LBF_arr = []
    for cord in range(len(labelled_BF_points)):
        x = labelled_BF_points[cord][0]
        y = labelled_BF_points[cord][1]
        z = 0
        LBF_arr.append([x,y,z])

    LBF_arr = np.array(LBF_arr)

    return LBF_arr

def Get_BuildingLidarPoints(BIDs_Arr, Building_coords_dict, lasdf):


    lidarPointsRaw = lasdf.iloc[:,:3].to_numpy()
    # Selecting Lidar Points within Building footprint

    X_max , X_min = lasdf.X.max(), lasdf.X.min()
    Y_max , Y_min = lasdf.Y.max(), lasdf.Y.min()

    X_plane_tile_divisor = 50 #in m - indicates the number of tiles you want to divide the tiles into
    Y_plane_tile_divisor = 50 #in m

    X_diff = X_max - X_min
    Y_diff = Y_max - Y_min

    X_div_len = X_diff/X_plane_tile_divisor
    Y_div_len = Y_diff/Y_plane_tile_divisor

    # print(X_max , X_min)
    # print(Y_max , Y_min)

    #NOTE : Iterating over all the points takes too long, need a more optimized way 

    lidar_BpointsDict = {}

    #For every building under the effect of the shadow
    for B_ID in BIDs_Arr:

        #List to append lidar points into for a specifc building
        B_ID_LidarPointsTempList = []

        #get the sampled building footprint points
        bf_points = Building_coords_dict[B_ID]
        

        #create a 2D convex Hull
        bf_shape = ConvexHull(bf_points[:,:2])

        #Get an area of points to look at (No need to iterate through all points in the tile)
        #get bounding subset to look at
        bf_shape_X_max = bf_shape.max_bound[0] + X_div_len
        bf_shape_Y_max = bf_shape.max_bound[1] + Y_div_len

        bf_shape_X_min = bf_shape.min_bound[0] - X_div_len
        bf_shape_Y_min = bf_shape.min_bound[1] - Y_div_len

        # print(bf_shape_X_max,bf_shape_Y_max,bf_shape_X_min,bf_shape_Y_min)

        #Bounded_Vertices_BF = bf_points[bf_shape.vertices]

        lidar_subset_df = lasdf[
            (lasdf['X'].between(bf_shape_X_min, bf_shape_X_max, inclusive=False) &
        lasdf['Y'].between(bf_shape_Y_min, bf_shape_Y_max, inclusive=False))
        ]

        # print("subset shape : ",lidar_subset_df.shape)


        #Get only points from BF
        lidar_BFMasked_points = lidar_subset_df.iloc[:,:3].to_numpy()

        #Check if a lidar point is within a building Footprint 

        #Add a z-coordinate to BF ,  needed for alphashape
        bf_points_with_Z = np.c_[bf_points, np.zeros(len(bf_points))]
        #Create a shape of the building footprint
        bf_shape_alpha = alphashape.alphashape(bf_points_with_Z, alpha=0)

        for p in lidar_BFMasked_points:
            
            tp = Point(p)
            if(tp.within(bf_shape_alpha)):
                B_ID_LidarPointsTempList.append(p)

        if B_ID not in lidar_BpointsDict:
            lidar_BpointsDict[B_ID] = np.array(B_ID_LidarPointsTempList)

    return lidar_BpointsDict

def Set_BuildingLidarHeightAdjustment(BuidlingDataDict, height_Adjustment):
    
    for B_ID, B_Lidarpoints in BuidlingDataDict.items():

        bpArr = B_Lidarpoints
        #adjust height of buildings
        lidar_Bpoints_Hadj = []
        for bp in bpArr:
            lidar_Bpoints_Hadj.append(bp - [0,0,height_Adjustment])
        
        BuidlingDataDict[B_ID] = np.array(lidar_Bpoints_Hadj)
    
    return BuidlingDataDict


#https://github.com/ulikoehler/UliEngineering/blob/master/UliEngineering/Math/Coordinates.py
# -*- coding: utf-8 -*-
"""
Estimating BOunding Box
"""
__all__ = ["BoundingBox"]

class BoundingBox(object):
    """
    A 2D bounding box
    """
    def __init__(self, points):
        """
        Compute the upright 2D bounding box for a set of
        2D coordinates in a (n,2) numpy array.

        You can access the bbox using the
        (minx, maxx, miny, maxy) members.
        """
        if len(points.shape) != 2 or points.shape[1] != 2:
            raise ValueError("Points must be a (n,2), array but it has shape {}".format(
                points.shape))
        if points.shape[0] < 1:
            raise ValueError("Can't compute bounding box for empty coordinates")
        self.minx, self.miny = np.min(points, axis=0)
        self.maxx, self.maxy = np.max(points, axis=0)

    @property
    def width(self):
        """X-axis extent of the bounding box"""
        return self.maxx - self.minx

    @property
    def height(self):
        """Y-axis extent of the bounding box"""
        return self.maxy - self.miny

    @property
    def area(self):
        """width * height"""
        return self.width * self.height

    @property
    def aspect_ratio(self):
        """width / height"""
        return self.width / self.height

    @property
    def center(self):
        """(x,y) center point of the bounding box"""
        return (self.minx + self.width / 2, self.miny + self.height / 2)

    @property
    def max_dim(self):
        """The larger dimension: max(width, height)"""
        return max(self.width, self.height)

    @property
    def min_dim(self):
        """The larger dimension: max(width, height)"""
        return min(self.width, self.height)

    def __repr__(self):
        return "BoundingBox({}, {}, {}, {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
    #NOTE: Added by me
    def Get_Coords(self):
        return [self.minx, self.maxx, self.miny, self.maxy]
    

def Get_TreeShadowBoundingBoxLimits(TreeShadow):
    Box_obj = BoundingBox(TreeShadow[:,:2])
    Box_Bounds = np.array(Box_obj.Get_Coords())

    B_minx = Box_Bounds[0]
    B_maxx = Box_Bounds[1]
    B_miny = Box_Bounds[2]
    B_maxy = Box_Bounds[3]

    return B_minx, B_maxx, B_miny, B_maxy

def Get_TreeShadowBoundingBoxesCoords(TreeShadow, Distance):

    B_minx, B_maxx, B_miny, B_maxy = Get_TreeShadowBoundingBoxLimits(TreeShadow)

    Tree_ShadowBox_Coords = [
        [B_minx, B_miny, 0],
        [B_minx, B_maxy, 0],
        [B_maxx, B_maxy, 0],
        [B_maxx, B_miny, 0],
        [B_minx, B_miny, 0] # final one is to complete the box used for plotting purposes only
    ]

    Extend_dist = Distance #in m
    Tree_ShadowBoxExtend_Coords = [
        [B_minx, B_miny - Extend_dist, 0],
        [B_minx, B_maxy, 0],
        [B_maxx, B_maxy, 0],
        [B_maxx, B_miny - Extend_dist, 0],
        [B_minx, B_miny - Extend_dist, 0] # final one is to complete the box used for plotting purposes only
    ]

    return Tree_ShadowBox_Coords, Tree_ShadowBoxExtend_Coords

#Facade shade
def Get_FacadeShadePoints(TreeShadow, PrimaryBuilding_IdArr, Building_coords_dict):

    FacadeShade_Points = []

    RemainingShadow_Points = TreeShadow

    for B_ID in PrimaryBuilding_IdArr:

        #get the sampled building footprint points
        bf_points = Building_coords_dict[B_ID]

        #Get only points from BF
        BuildingFootprint_shape = alphashape.alphashape(bf_points, alpha=0)

        curr_BF_bool_list = []
        #find Tree Shadow Points in StreetShade
        for tsp in RemainingShadow_Points:
            tp = Point(tsp)
            curr_BF_bool_list.append(tp.within(BuildingFootprint_shape))
        
        for fp in RemainingShadow_Points[curr_BF_bool_list]: #only inside the BF shape
            FacadeShade_Points.append(fp)
        
        RemainingShadow_Points = RemainingShadow_Points[~np.array(curr_BF_bool_list)]
    
    return RemainingShadow_Points, FacadeShade_Points

def Get_InShadePoints(TreeShadow, All_Building_LidarPointsDict_HAdj, Az, Amp):

    InShade_Points = []
    RemainingShadow_Points = TreeShadow

    for B_ID, B_lidarpoints in All_Building_LidarPointsDict_HAdj.items():

        B_Shadow = Get_Shadow(B_lidarpoints, Az, Amp)

        #Get only points from BS
        BuildingShadow_shape = alphashape.alphashape(B_Shadow, alpha=0)

        curr_BS_bool_list = []
        #find Tree Shadow Points in InShade
        for tsp in RemainingShadow_Points:
            tp = Point(tsp)
            curr_BS_bool_list.append(tp.within(BuildingShadow_shape))
        
        for Ip in RemainingShadow_Points[curr_BS_bool_list]: #only inside the BS shape
            InShade_Points.append(Ip)
            
        if(len(curr_BS_bool_list) > 0):
            RemainingShadow_Points = RemainingShadow_Points[~np.array(curr_BS_bool_list)]
    
    return RemainingShadow_Points, InShade_Points

def InitiateShadingLogger(filename:str,year:int)-> None:

    #Where the logger file will be created
    LoggerPath = "Datasets/"+"Package_Generated/"+filename[:-4]+"/"+str(year)+"/Shading_Logs_"+filename[:-4]+"/"

    print("Logger Folder Path : ",LoggerPath)
    # Check whether the specified LoggerPath exists or not
    isExist = os.path.exists(LoggerPath)

    if not isExist:
    # Create a new directory because it does not exist 
        os.makedirs(LoggerPath)

    logfilename = LoggerPath + 'ShadingScript_'+filename[:-4]+'.log' 
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=logfilename, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.INFO)

def parse_log_Completedfiles(log_file_path):
    """_summary_

    Args:
        log_file_path (_type_): _description_

    Returns:
        _type_: _description_
    
    Usage:
    log_file_path = 'TEST_log.log'
    filenames = parse_log_Completedfiles(log_file_path)
    print(filenames)
    """
    filenames = []
    with open(log_file_path, 'r') as log_file:
        for line in log_file:
            match = re.search(r'(?<=Completed file: )\S+', line)
            if match:
                filenames.append(match.group())
    return filenames

def Get_dirnames(parent_path:str)->list:
    dirnames = [d for d in os.listdir(parent_path) if os.path.isdir(os.path.join(parent_path, d))]
    return dirnames


In [2]:
#NOTE:
"""
Potential count of shading files
24 hrs at 15 min intervals - 24 x 4
Count of trees on average - 2500 - 500(not park)
Number of lidar files in 2017 - 2000
year round = 365 x 24 x 4 x 2500(including parks) x 2000 = 1.752e^11 rows - BIG DATA :D?
"""

'\nPotential count of shading files\n24 hrs at 15 min intervals - 24 x 4\nCount of trees on average - 2500 - 500(not park)\nNumber of lidar files in 2017 - 2000\nyear round = 365 x 24 x 4 x 2500(including parks) x 2000 = 1.752e^11 rows - BIG DATA :D?\n'

# Functions rewritten (NEW JSON Structure)

In [3]:
# Reading JSON tree data
def Get_JSONfilenames(folder_path:str, year:int)->list:
    filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return filenames


def Get_TreeLocation(json_dataBuffer):
    lat = json_dataBuffer['PredictedTreeLocation']['Latitude']
    lon = json_dataBuffer['PredictedTreeLocation']['Longitude']
    return lat,lon

def Get_TreeGroundElevation(json_dataBuffer):
    return json_dataBuffer['GroundZValue']

def Get_TreeHeight(json_dataBuffer):
    return json_dataBuffer['TreeFoliageHeight']

def Get_FullTree_points(json_dataBuffer, Ground_Elevation = 0, Tree_Height = 0):
    Full_Tree = []
    
    #TODO : might be doing repetitive conversion here
    All_SingleTree_points = np.array(json_dataBuffer['Tree_Points'])*3.28 # Convert m to ft
    for p in All_SingleTree_points:
        Full_Tree.append(p - [0,0,Ground_Elevation+Tree_Height]) #Height Adjusted, default = 0
            
    return np.array(Full_Tree)

def Get_TreesNotPark(json_filenames, JSON_TreeData_path):
    JsonFiles_notInPark = []
    for jf in json_filenames:
        json_filobj = open(JSON_TreeData_path + jf)
        json_SingleTreeDataBuffer = json.load(json_filobj)
        if not json_SingleTreeDataBuffer['InPark']:
            JsonFiles_notInPark.append(jf)
    
    return JsonFiles_notInPark

def Get_AllTreePoints_from_JSONfiles(json_filenames,JSON_TreeData_path):
    tree_p = []
    for jf in json_filenames:
        json_filobj = open(JSON_TreeData_path + jf)
        json_SingleTreeDataBuffer = json.load(json_filobj)

        singleTreePoints = json_SingleTreeDataBuffer['Tree_Points']

        for p in singleTreePoints:
            tree_p.append(p)
    
    return tree_p

# def Get_SampledBuildingFootprints(Buildingdata):
#     BuildingFeature_Coords = [np.array(F['geometry']['coordinates'][0]) for F in Buildingdata['features']]
#     SampledFootprint = np.vstack([sample_polygon(W) for W in BuildingFeature_Coords])
#     SampledFootprint = np.c_[SampledFootprint, np.zeros(len(SampledFootprint))]

#     return SampledFootprint

# Functions to Trim GeoJSON for current tile

In [4]:
def footprintPointsFromGeoJSON_v2(feature):   
    points = []

    try:
        height = feature["properties"]["heightroof"] 
        
        if type(height) == float:
            height = height
        else:
            height = 0
    except:
        height = 0
    
    if feature['geometry']['type'] == 'Point':
        point = [feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1],height]
        points.append(point)
        point = [feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1],0]
        points.append(point) 
    elif feature['geometry']['type'] == 'LineString':
        for polygonPart in feature["geometry"]["coordinates"]:                
            for polygonSubPart in polygonPart:
                point = [polygonSubPart[0],polygonSubPart[1],height]
                points.append(point)
                point = [polygonSubPart[0],polygonSubPart[1],0]
                points.append(point)                  
    # elif feature['geometry']['type'] == 'Polygon':
    #     for polygonPart in feature["geometry"]["coordinates"]:                
    #         for polygonSubPart in polygonPart:
    #             for coordinates in polygonSubPart:
    #                 point = [coordinates[0],coordinates[1],height]
    #                 points.append(point)
    #                 point = [coordinates[0],coordinates[1],0]
    #                 points.append(point)
    elif feature['geometry']['type'] == 'MultiPolygon':
        for MpolygonPart in feature["geometry"]["coordinates"]:
            for polygonPart in MpolygonPart:                
                for polygonSubPart in polygonPart:
                    #print(polygonSubPart)
                    #for coordinates in polygonSubPart:
                    point = [polygonSubPart[0],polygonSubPart[1],height]
                    points.append(point)
                    point = [polygonSubPart[0],polygonSubPart[1],0]
                    points.append(point) 
    else:
        pass
        #print(feature['geometry']['type'])
        #print(feature["geometry"]["coordinates"])

    return points, height

def findCentroid(buildingPoints):
    xs = []
    ys = []
    for buildingPoint in buildingPoints:
        xs.append(buildingPoint[0])
        ys.append(buildingPoint[1])
    xCenter = sum(xs)/len(xs)
    yCenter = sum(ys)/len(ys)
    return xCenter, yCenter

def convertLatLon(lat,lon):
    #translate from geojson CRS (NAD 1983) to .las CRS (UTM Zone 18N (meters))
    transformer = Transformer.from_crs( "epsg:4326", "epsg:2263" ) 
    x, y = transformer.transform(lat, lon)
    return x, y

def convertXY(x,y):
    #translate from .las CRS to geojson CRS (NAD 1983) (UTM Zone 18N (meters))
    transformer = Transformer.from_crs( "epsg:2263", "epsg:4326" ) 
    lat, lon = transformer.transform(x, y)
    return lat, lon

def lasdf_bounds(lasdf):

    xMin = min(lasdf.X)
    xMax = max(lasdf.X)
    yMin = min(lasdf.Y)
    yMax = max(lasdf.Y)

    return xMin,yMin,xMax,yMax

def trimGeoJSON_v2(features,lasdf):

    #print(len(features))

    xMin,yMin,xMax,yMax = lasdf_bounds(lasdf)
    
    Min_lat,Min_lon  = convertXY(xMin, yMin)
    Max_lat,Max_lon  = convertXY(xMax, yMax)


    features2 = []
    
    for feature in features: #NOTE: modified from - for feature in features[:]:
        buildingPoints,height = footprintPointsFromGeoJSON_v2(feature)

        xCenter, yCenter = findCentroid(buildingPoints)

        #lat varies in y direction
        #lon varies in x direction
        
        if yCenter > Min_lat and yCenter < Max_lat and xCenter > Min_lon and xCenter < Max_lon:
            features2.append(feature)
        
        else:
            continue
        
    return features2
    
def convert_dict_of_arrays(data):
    transformer = Transformer.from_crs("epsg:4326", "epsg:2263" )
    converted_data = {}
    for key, value in data.items():
        x, y = transformer.transform(value[:,1], value[:,0])
        converted_data[key] = np.column_stack((x, y))
    return converted_data

# def convert_coordinates(data): #convert all lat,lon values to geojson values
#     for feature in data:
#         geometry = feature['geometry']
#         coordinates = geometry['coordinates'][0][0]
#         new_coordinates = []
#         for point in coordinates:
#             x, y = point[0], point[1]
#             lat, lon = convertXY(x, y)
#             new_coordinates.append([lon, lat])
#         geometry['coordinates'] = [new_coordinates]
#     return data

In [5]:
#TODO: Add for loop to interate over lasfiles
f = '25192.las'
year = 2017

In [6]:
script_start_time = time.time()

#def ProcessShading(f, year): #ProcessShading(f, year, lasParent_folderpath)

InitiateShadingLogger(f,year)

# Extracting individual Tilesets from NYC Building Footprints GeoJSON

#Approx 40 seconds - 1,150,733 buildings
NYC_GeoJSON_Filepath = 'Datasets/BuildingTileSet/building_BL.geojson'
BuidlingFootprintData = Read_GeoJSON(NYC_GeoJSON_Filepath)

las_file_path = 'Datasets/Package_Generated/'+f[:-4]+'/'+str(year)+'/LasClassified_'+f[:-4]+'/lasFile_Reconstructed_'+f[:-4]+'.las'

logging.info("Reading las file from : %s",las_file_path)

#Create a dataframe from las file
lasdf = Create_Dataframe_fromLas(las_file_path)

lasdf.X = lasdf.X*3.28
lasdf.Y = lasdf.Y*3.28
lasdf.Z = lasdf.Z*3.28

Logger Folder Path :  Datasets/Package_Generated/25192/2017/Shading_Logs_25192/


In [7]:
Filtered_FeatureList = trimGeoJSON_v2(BuidlingFootprintData['features'],lasdf)

#Save to Geojson

# Serialize features into a GeoJSON string
features_geojson = {
    "type": "FeatureCollection",
    "features": Filtered_FeatureList
}
features_str = json.dumps(features_geojson)

geojson_folderpath = 'Datasets/Package_Generated/'+f[:-4]+'/'+str(year)+'/GeoJSON_BuildingFootprint_'+f[:-4]+'/'
geojson_filename = 'Filtered_buildingsTile_'+f[:-4]+'.geojson'

# Check whether the specified geojson_folderpath exists or not
isExist = os.path.exists(geojson_folderpath)

if not isExist:
# Create a new directory because it does not exist 
    os.makedirs(geojson_folderpath)
# Write GeoJSON string to a file
with open(geojson_folderpath+geojson_filename, "w") as file:
    file.write(features_str)
    logging.info("Building Footprint for tileset created")


In [8]:
Building_coords_dict_latlong_coords = Get_BuildingDataDict(geojson_folderpath+geojson_filename, f, year=2017)
Building_coords_dict = convert_dict_of_arrays(Building_coords_dict_latlong_coords)

In [9]:
logging.info("TerraVide lidar Shading Metrics Estimation Initated")
# #To scale
# las_file_path = 'Datasets/Package_Generated/'+f[:-4]+'/'+str(year)+'/LasClassified_'+f[:-4]+'/lasFile_Reconstructed_'+f[:-4]+'.las'

# logging.info("Reading Reconstructed lasfile from : %s",las_file_path)

# #Create a dataframe from las file
# lasdf = Create_Dataframe_fromLas(las_file_path)

# lasdf.X = lasdf.X*3.28
# lasdf.Y = lasdf.Y*3.28
# lasdf.Z = lasdf.Z*3.28

#Path to where each lasfiles JSON tree data is located (generated by Process_lasFiles.py)
JSON_TreeData_path = 'Datasets/Package_Generated/'+f[:-4]+'/'+str(year)+'/JSON_TreeData_'+f[:-4]+'/'
#Get filenames(1 json file = 1 tree cluster) of each json file
JSON_filenames = Get_JSONfilenames(JSON_TreeData_path,year)
#Get trees not present in park areas
JSON_filenamesNotPark = Get_TreesNotPark(JSON_filenames, JSON_TreeData_path)

logging.info("Total number of Trees : %d",len(JSON_filenames))
logging.info("Total number of Trees not in Park: %d",len(JSON_filenamesNotPark))

#Used for debugging with pptk viewer
Raw_TreePointsNotPark = Get_AllTreePoints_from_JSONfiles(JSON_filenamesNotPark,JSON_TreeData_path)

# CSV header
header = ['DateTime_ISO', 'Year', 'Month', 'Day', 'hour', 'minute', 'Sun_Azimuth', 'Sun_Amplitude',
    'Tree_Number', 'Tree_Latitude', 'Tree_Longitude',
    'Shadow_Length','Shadow_Breadth','Shadow_Area','TreeShadow_PointCount',
    'Perc_Canopy_StreetShade', 'Perc_Canopy_FacadeShade', 'Perc_Canopy_InShade',
    'ShadowArea_Ground', 'ShadowArea_OnBuilding', 'ShadowArea_InBuildingShadow']

In [10]:
#Iterating over 10 trees in tile 25192 which are not in park
for jf in JSON_filenamesNotPark[:10]: #10 trees
    #Reading a single file
    TreeId = jf.split('_')[3] #25192_2017_ID_1_TreeCluster.json
    
    # tree_ID = sorted_dict[list(sorted_dict.keys())[tree_index]]
    # tree_ID = 37
    # json_filobj = open(JSON_TreeData_path + JSON_filenames[tree_ID])
    json_filobj = open(JSON_TreeData_path + jf)
    json_SingleTreeDataBuffer = json.load(json_filobj)

    shadingCSV_folderpath = 'Datasets/Package_Generated/'+f[:-4]+'/'+str(year)+'/ShadingMetrics'+f[:-4]+'/'
    shadingCSV_filename = 'ShadingMetric_'+f[:-4]+'_Tree_ID_'+TreeId+'.csv'

    # Check whether the specified geojson_folderpath exists or not
    isExist = os.path.exists(shadingCSV_folderpath)

    if not isExist:
    # Create a new directory because it does not exist 
        os.makedirs(shadingCSV_folderpath)

    with open(shadingCSV_folderpath+shadingCSV_filename, 'w', encoding='UTF8') as csv_f:
        writer = csv.writer(csv_f)

        # write the header
        writer.writerow(header)

        #test - where is the tree selected located

        # p1 = Raw_TreePointsNotPark
        # p2 = json_SingleTreeDataBuffer['Tree_Points']
        # All_points_1 = np.concatenate((p1, p2), axis=0)
        # rgb_p1 =  [[1,0,0]]*len(p1) #Set red colour
        # rgb_p2 = [[255,255,255]]*len(p2) #set green colour - Classified tree points
        # All_rgb = np.concatenate((rgb_p1, rgb_p2,), axis=0)

        # v = pptk.viewer(All_points_1, All_rgb)
        # v.set(show_grid=False)
        # v.set(show_axis=False)
        # v.set(bg_color = [0,0,0,1])
        # v.set(point_size = 0.04)
        # time.sleep(1)
        # v.close()

        # DATE
        # year = 2021 #DATE_DICT['YEAR']
        # month = 6 #DATE_DICT['MONTH']
        # day = 21 #DATE_DICT['DAY']
        # #specific hour and minute
        # hour = 8
        # minute = 30

        #taking rougly 10 days in Jan for testing
        #Total number of files - 575 trees * 10 days * 24 hrs * 4 intervals
        # Testing for 10 trees - 1 day
        shadeMyear = 2021
        start = datetime.datetime(shadeMyear, 1, 1)
        end = datetime.datetime(shadeMyear, 1, 1, 23, 59)
        delta = timedelta(minutes=15)


        current = start
        while current <= end:
            day = current.day
            hour = current.hour
            minute = current.minute
            month = current.month
            # Use the values as needed
            current += delta

            #print(shadeMyear, month, day, hour, minute)

            #Tree Data
            Tree_Lat , Tree_Long = Get_TreeLocation(json_SingleTreeDataBuffer)
            Tree_GroundZ = Get_TreeGroundElevation(json_SingleTreeDataBuffer) #Convert to ft, JSON file has m stored
            Tree_Height = Get_TreeHeight(json_SingleTreeDataBuffer)

            # GET AZ and Amp
            date, Az, Amp = Get_SunData(Tree_Lat, Tree_Long, shadeMyear, month, day, hour, minute)

            if Amp > 0:
                #Get Tree Points
                Full_Tree = Get_FullTree_points(json_SingleTreeDataBuffer,Tree_GroundZ)

                #Proj_Tree = get_projection(Full_Tree, 45, True)
                TreeShadow = Get_Shadow(Full_Tree, Az, Amp) #(points, az, amp)

                L,B,A = Get_ShadowCharacteristics(Full_Tree, TreeShadow)

                # BuildingFilePath = 'Datasets/BuildingTileSet/buildingsTile25192.geojson' 
                # #'Datasets/Package_Generated/25192/2017/GeoJSON_BuildingFootprint_25192/buildingsTile25192.geojson'

                # BuildingFootprints = Get_BuildingFootprint(BuildingFilePath)
                # Building_coords_dict = Get_BuildingDataDict(BuildingFilePath)

                #Get Primary Building IDs
                #PrimaryBuilding_IdArr = Get_BuildingsUnderEffectOfShadow(Building_coords_dict, TreeShadow)
                PrimaryBuilding_IdArr = Get_BuildingsUnderEffectOfShadow_Rapid(Building_coords_dict, TreeShadow) #Function is 50% faster
                #Get Primary Building Coords
                PrimaryBuilding_FootprintCoords = Get_BuildingFootprintSampledCoords(PrimaryBuilding_IdArr, Building_coords_dict)

                #Get Primary Building Lidar Points
                PrimaryBuilding_LidarPointsDict = Get_BuildingLidarPoints(PrimaryBuilding_IdArr, Building_coords_dict, lasdf)

                #Get ground elevation from building points
                G_height = Tree_GroundZ #NOTE : Same as Tree_GroundZ
                #Adjusting height of Buildings
                PrimaryBuilding_LidarPointsDict_HAdj = Set_BuildingLidarHeightAdjustment(PrimaryBuilding_LidarPointsDict, G_height)

                #Bounding Box
                Distance = 200
                Tree_ShadowBox_Coords, Tree_ShadowBoxExtend_Coords = Get_TreeShadowBoundingBoxesCoords(TreeShadow, Distance)
                TSBox = sample_polygon(Tree_ShadowBox_Coords)
                TSBox = np.c_[TSBox, np.zeros(len(TSBox))]
                TSBoxExtend = sample_polygon(Tree_ShadowBoxExtend_Coords)
                TSBoxExtend = np.c_[TSBoxExtend, np.zeros(len(TSBoxExtend))]

                #Get All buildings Falling under Extended Tree ShadowBox
                All_Building_IdArr = Get_BuildingsUnderEffectOfShadow_Rapid(Building_coords_dict, TSBoxExtend)
                #Get Secondary Building Lidar Points
                All_Building_LidarPointsDict = Get_BuildingLidarPoints(All_Building_IdArr, Building_coords_dict, lasdf)
                #Get ground elevation from building points
                G_height = Tree_GroundZ #NOTE : Same as Tree_GroundZ
                #Adjusting height of Buildings
                All_Building_LidarPointsDict_HAdj = Set_BuildingLidarHeightAdjustment(All_Building_LidarPointsDict, G_height)

                #Get Secondary Buildings
                SecondaryBuilding_IdArr = [b for b in All_Building_IdArr if b not in PrimaryBuilding_IdArr]
                #Get Secondary Building Coords
                SecondaryBuilding_FootprintCoords = Get_BuildingFootprintSampledCoords(SecondaryBuilding_IdArr, Building_coords_dict)
                #Get Secondary Building Lidar Points
                SecondaryBuilding_LidarPointsDict = Get_BuildingLidarPoints(SecondaryBuilding_IdArr, Building_coords_dict, lasdf)
                #Get ground elevation from building points
                G_height = Tree_GroundZ #NOTE : Same as Tree_GroundZ
                #Adjusting height of Buildings
                SecondaryBuilding_LidarPointsDict_HAdj = Set_BuildingLidarHeightAdjustment(SecondaryBuilding_LidarPointsDict, G_height)

                # Shade Metrics

                # 1. Facade Shade - Points falling on the building footprint
                # 2. In Shade - Points Fall within the shadow of Buildings not in the building footprint
                # 3. Street Shade - Tree Shadow Points not overcasted by Anything
                RemainingShadow_Points, FacadeShade_Points = Get_FacadeShadePoints(TreeShadow, PrimaryBuilding_IdArr, Building_coords_dict)
                RemainingShadow_Points, InShade_Points = Get_InShadePoints(RemainingShadow_Points, All_Building_LidarPointsDict_HAdj, Az, Amp)
                StreetShade_Points = RemainingShadow_Points

                #print(len(FacadeShade_Points)/len(TreeShadow) , len(InShade_Points)/len(TreeShadow) ,len(StreetShade_Points)/len(TreeShadow))

                Perc_Canopy_StreetShade = round(len(StreetShade_Points)/len(TreeShadow),3)
                Perc_Canopy_FacadeShade = round(len(FacadeShade_Points)/len(TreeShadow),3)
                Perc_Canopy_InShade = round(len(InShade_Points)/len(TreeShadow),3)

                data = [date,shadeMyear, month, day, hour+4, minute, Az, Amp,
                        TreeId, round(Tree_Lat,6), round(Tree_Long,6), round(L,3), round(B,3), round(A,3), len(TreeShadow),
                        Perc_Canopy_StreetShade*100, Perc_Canopy_FacadeShade*100, Perc_Canopy_InShade*100,
                        A*Perc_Canopy_StreetShade, A*Perc_Canopy_FacadeShade, A*Perc_Canopy_InShade]
            else:
                data = [date,shadeMyear, month, day, hour+4, minute, Az, Amp,
                        TreeId, round(Tree_Lat,6), round(Tree_Long,6), 0, 0, 0, 0,
                        0, 0, 0,
                        0, 0, 0]

            print(data)
            writer.writerow(data)

0.0 0.0 1.0
['2020-12-31T19:00:00-05:00', 2021, 1, 1, 4, 0, 261.1802658552883, -26.042087296173445, '3180', 40.70178023482098, -73.84911620303673, 333.13668464517315, 1.1147781847283922, 371.37350857518095, 530, 100.0, 0.0, 0.0, 371.37350857518095, 0.0, 0.0]
0.0 0.0 1.0
['2020-12-31T19:15:00-05:00', 2021, 1, 1, 4, 15, 263.42851004261917, -28.857743229953027, '3180', 40.70178023482098, -73.84911620303673, 296.3767340769187, 1.1338275588519222, 336.040108898938, 530, 100.0, 0.0, 0.0, 336.040108898938, 0.0, 0.0]
0.0 0.0 1.0
['2020-12-31T19:30:00-05:00', 2021, 1, 1, 4, 30, 265.71758171303327, -31.686247593319806, '3180', 40.70178023482098, -73.84911620303673, 265.4650911174768, 1.1539844845995797, 306.3425963523819, 530, 100.0, 0.0, 0.0, 306.3425963523819, 0.0, 0.0]
0.0 0.0 1.0
['2020-12-31T19:45:00-05:00', 2021, 1, 1, 4, 45, 268.0630526950327, -34.52329787999844, '3180', 40.70178023482098, -73.84911620303673, 239.0130649663209, 1.1755815180680156, 280.9793417511968, 530, 100.0, 0.0, 0.0, 

KeyboardInterrupt: 

In [ ]:
script_end_time = time.time()

print("SCRIPT TOTAL TIME (in min): ",(script_end_time - script_start_time)/60)

In [ ]:
# import numpy as np
# 
# from functools import partial

# def convert_xy(x, y):
#     transformer = pyproj.Transformer.from_crs("epsg:2263", "epsg:4326")
#     return transformer.transform(x, y)

# def convert_coordinates(coordinates):
#     x, y = np.array(coordinates).T
#     lat, lon = convert_xy(x, y)
#     return np.column_stack((lat, lon)).tolist()

# def convert_features(features):
#     for feature in features:
#         geometry = feature['geometry']
#         if geometry['type'] == 'MultiPolygon':
#             geometry['coordinates'] = [
#                 [convert_coordinates(polygon) for polygon in polygons]
#                 for polygons in geometry['coordinates']
#             ]
#         elif geometry['type'] == 'Polygon':
#             geometry['coordinates'] = [convert_coordinates(polygon) for polygon in geometry['coordinates']]
#     return features

# def convert_geojson(geojson):
#     return {
#         'type': 'FeatureCollection',
#         'features': convert_features(geojson['features'])
#     }


In [ ]:
# test = convert_geojson(Filtered_BuildingFootprints)

In [ ]:
#Script taking too long
#test = convert_coordinates(Filtered_BuildingFootprints['features'])

In [ ]:
# len(Filtered_FeatureList)

In [ ]:
# for feature in  Filtered_FeatureList['feature']
#     if feature['geometry']['type'] == 'Point':
#         print('point')
#         #feature["geometry"]["coordinates"][0], feature["geometry"]["coordinates"][0] = convertLatLon(feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1])
#     elif feature['geometry']['type'] == 'LineString':
#         print('line')
#         # for polygonPart in feature["geometry"]["coordinates"]:                
#         #     for polygonSubPart in polygonPart:
#         #         feature["geometry"]["coordinates"][0], feature["geometry"]["coordinates"][0] = convertLatLon(feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1])
#     elif feature['geometry']['type'] == 'Polygon':
#         for polygonPart in feature["geometry"]["coordinates"]:                
#             for polygonSubPart in polygonPart:
#                 for coordinates in polygonSubPart:
#                     feature["geometry"]["coordinates"][0], feature["geometry"]["coordinates"][0] = convertLatLon(feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1])
#     elif feature['geometry']['type'] == 'MultiPolygon':
#         for MpolygonPart in feature["geometry"]["coordinates"]:
#             for polygonPart in MpolygonPart:                
#                 for polygonSubPart in polygonPart:
#                     feature["geometry"]["coordinates"][0], feature["geometry"]["coordinates"][0] = convertLatLon(feature["geometry"]["coordinates"][0],feature["geometry"]["coordinates"][1])

In [ ]:
# Gen_las = laspy.read(las_file_path)

# Xscale = Gen_las.header.x_scale
# Yscale = Gen_las.header.y_scale
# Zscale = Gen_las.header.z_scale

# Xoffset = Gen_las.header.x_offset
# Yoffset = Gen_las.header.y_offset
# Zoffset = Gen_las.header.z_offset

# Gen_lidarpoints = np.array(
#     ( (Gen_las.X*1.00) + Xoffset,  # convert ft to m and correct measurement
#     (Gen_las.Y*1.00) + Yoffset,
#     (Gen_las.Z*1.00) + Zoffset,
#     Gen_las.intensity,
#     Gen_las.classification,
#     Gen_las.return_number, 
#     Gen_las.number_of_returns)).transpose()
# G_lidar_df = pd.DataFrame(Gen_lidarpoints , columns=['X','Y','Z','intensity','classification','return_number','number_of_returns'])

# G_las_Gpoints = G_lidar_df.iloc[:,:3][G_lidar_df["classification"] == 1].to_numpy()
# G_las_Tpoints = G_lidar_df.iloc[:,:3][G_lidar_df["classification"] == 2].to_numpy()
# G_las_SRpoints = G_lidar_df.iloc[:,:3][G_lidar_df["classification"] == 3].to_numpy()
# G_las_NTpoints = G_lidar_df.iloc[:,:3][G_lidar_df["classification"] == 4].to_numpy()
# G_las_NCpoints = G_lidar_df.iloc[:,:3][G_lidar_df["classification"] == 5].to_numpy()

# #plotting inlier and outlier
# All_points_1 = np.concatenate((G_las_Gpoints, G_las_Tpoints,G_las_SRpoints,G_las_NTpoints,G_las_NCpoints), axis=0)
# rgb_Ground =  [[1,0,0]]*len(G_las_Gpoints) #Set red colour
# rgb_Tree = [[0,1,0]]*len(G_las_Tpoints) #set green colour
# rgb_SR = [[0,0,1]]*len(G_las_SRpoints) #set blue colour
# rgb_NT = [[255,255,255]]*len(G_las_NTpoints) #set white colour
# rgb_NC = [[255,255,0]]*len(G_las_NCpoints) #set cyan colour
# All_rgb = np.concatenate((rgb_Ground, rgb_Tree,rgb_SR,rgb_NT,rgb_NC), axis=0)

# #Red - Inlier - ground plane , Green - Outlier
# v = pptk.viewer(All_points_1, All_rgb)
# v.set(show_grid=False)
# v.set(show_axis=False)
# v.set(bg_color = [0,0,0,1])
# v.set(point_size = 0.1)

In [ ]:
# Below block was used to sort tree cluster by their tree ids and index in JSON_filename array for ease of debugging
# Use when needed

# TreeID_filename_map = {} #filename -> index in JSON_filenamesNotPark
# for i,jf in enumerate(JSON_filenamesNotPark):
#     tree_id = jf.split('_')[3]
#     if tree_id not in TreeID_filename_map:
#         TreeID_filename_map[tree_id] = i

# #sort by keys
# myKeys = list(TreeID_filename_map.keys())
# myKeys.sort()
# sorted_dict = {i: TreeID_filename_map[i] for i in myKeys}



In [ ]:
#pptk.viewer(json_SingleTreeDataBuffer['Tree_Points']) #max(np.array(json_SingleTreeDataBuffer['Tree_Points'])[:,2])

In [ ]:
# #print values
# JSON_filenames[tree_ID], json_SingleTreeDataBuffer.keys(), json_SingleTreeDataBuffer['Tree_CountId'], json_SingleTreeDataBuffer['InPark'],json_SingleTreeDataBuffer['PredictedTreeLocation'],\
# json_SingleTreeDataBuffer['GroundZValue']

In [ ]:
#BuildingFootprints.shape,Full_Tree.shape,TreeShadow.shape,B_lidarP.shape, B_lidarS.shape, np.array(FacadeShade_Points).shape, np.array(InShade_Points).shape

In [ ]:
# #NOTE : Ensure all values are populated before plot

# #commented for plot test
# B_lidarP = (np.concatenate(list(PrimaryBuilding_LidarPointsDict_HAdj.values())))
# B_lidarS = (np.concatenate(list(SecondaryBuilding_LidarPointsDict_HAdj.values())))

# #ALL_points = np.concatenate((BuildingFootprints,Full_Tree,TreeShadow,B_lidarP, B_lidarS, FacadeShade_Points, InShade_Points, StreetShade_Points), axis=0)
# ALL_points_test = np.concatenate((BuildingFootprints,Full_Tree,TreeShadow,B_lidarP, B_lidarS), axis=0)

# rgb_BuildingFootprints =  [[255,255,255]]*len(BuildingFootprints) #Set red colour
# rgb_Full =  [[1,0,0]]*len(Full_Tree) #Set red colour
# rgb_Proj = [[0,1,0]]*len(TreeShadow) #set green colour
# rgb_B_lidarP = [[150,220,60]]*len(B_lidarP) 
# rgb_B_lidarS = [[200,200,0]]*len(B_lidarS) 
# rgb_FacadeShadePoints  = [COLOR_DICT['YELLOW']]*len(FacadeShade_Points)
# rgb_InShadePoints = [COLOR_DICT['LIGHT_BLUE']]*len(InShade_Points)
# rgb_StreetShadePoints = [COLOR_DICT['PINK']]*len(StreetShade_Points)
# #All_rgb = np.concatenate((rgb_BuildingFootprints,rgb_Full, rgb_Proj,rgb_B_lidarP,rgb_B_lidarS, rgb_FacadeShadePoints, rgb_InShadePoints, rgb_StreetShadePoints), axis=0)
# All_rgb_test = np.concatenate((rgb_BuildingFootprints,rgb_Full, rgb_Proj,rgb_B_lidarP,rgb_B_lidarS), axis=0)

# #Red - MR , Green - SR
# v = pptk.viewer(ALL_points_test, All_rgb_test)
# v.set(show_grid=False)
# v.set(show_axis=False)
# v.set(bg_color = [0,0,0,0])
# v.set(point_size = 0.01)

---
---
---

In [ ]:
#TEST SCRIPT - LOGS in TEST Directory - modify to Dataset paths with original files for deployment



#Get las directory names from parent root folder
p_rootpath = "Datasets/Package_Generated/" #'/Volumes/Elements/TerraVide/Datasets/FTP_files/LiDAR/'
year = 2017

LAS_dirnames = Get_dirnames(p_rootpath)

print("FileCount : "+str(len(LAS_dirnames))+" .las files found in path = "+p_rootpath )

args = [(i, year,p_rootpath) for i in LAS_dirnames]#testing for first 2 files

print(args)

# with Pool(2) as p:

#         p.starmap(ProcessShading,args)
    

In [ ]:
# from multiprocessing import Process, Manager

# with Manager() as manager:
#     Filtered_FeatureList = manager.list()  # <-- can be shared between processes.
#     processes = []
#     for i in range(10):
#         p = Process(target=trimGeoJSON_v3, args=(Filtered_FeatureList,BuidlingFootprintData['features'],lasdf,'latLon'))  # Passing the list
#         p.start()
#         processes.append(p)
#     for p in processes:
#         p.join()
#     #print(Filtered_FeatureList)

# from rtree import index as rt_index

# idx = rt_index.Index()
# k = 1084020 #tesing
# for i, feature in enumerate(BuidlingFootprintData["features"][k:]): #1084520
#     if (feature["geometry"]["type"]) == 'MultiPolygon':
#         for MpolygonPart in feature["geometry"]["coordinates"]:
#             for polygonPart in MpolygonPart:                
#                 for polygonSubPart in polygonPart:
#                     #for coordinates in polygonSubPart: 
#                     coords = polygonSubPart
#     else:
#         coords = feature["geometry"]["coordinates"]
    
#     #x,y = convertLatLon(coords[0],coords[1]) - this function takes time
#     idx.insert(i, (coords[0], coords[1], coords[0], coords[1]))

# [ #MultiPolygon
#     [ #Polygon
#         [ #LineString
#             [#Point
#               -73.96664570466969, 40.62599676998366
#             ], 
#             [-73.96684846176461, 40.625977490862574], 
#             [-73.96685938726297, 40.62604419372411], 
#             [-73.96661621040211, 40.62606731716107],
#             [-73.96660638332114, 40.626007324369795], 
#             [-73.96664680403327, 40.626003480977275], 
#             [-73.96664570466969, 40.62599676998366]
#         ]
#     ]
# ]